<a href="https://colab.research.google.com/github/stelmanj/MusicAndLanguage/blob/master/TTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#install.packages("quanteda")

In [0]:
library(dplyr)
library(quanteda)
library(stringi)

In [8]:
AFsongDF <- read.csv(
  "https://github.com/stelmanj/MusicAndLanguage/blob/master/AFsongDF.csv?raw=true",
  row.names = 1) 
head(AFsongDF)

Package version: 2.0.1

Parallel computing: 2 of 2 threads used.

See https://quanteda.io for tutorials and examples.


Attaching package: ‘quanteda’


The following object is masked from ‘jupyter:irkernel’:

    View


The following object is masked from ‘package:utils’:

    View




,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,⋯,track_href,analysis_url,duration_ms,time_signature,title,artist,lyrics,lang,langs,sid
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
0,0.228,0.739,2,-27.313,0,0.0555,0.3280,9.90e-01,0.4070,0.0112,⋯,https://api.spotify.com/v1/tracks/32AWEAYwQRky8gkAqKBgv1,https://api.spotify.com/v1/audio-analysis/32AWEAYwQRky8gkAqKBgv1,63869,5,Sharp Nighttime Thunder Storms Sounds,Thunderstorm Sound Bank,no lyrics found,NA,NA,32AWEAYwQRky8gkAqKBgv1
1,0.708,0.608,10,-8.780,0,0.3200,0.0476,7.15e-06,0.1270,0.2200,⋯,https://api.spotify.com/v1/tracks/3Cqj0VmGsjhmHgqXcnH6Zz,https://api.spotify.com/v1/audio-analysis/3Cqj0VmGsjhmHgqXcnH6Zz,228625,4,Pse Ke Ardh Ktu,Buta,no lyrics found,NA,NA,3Cqj0VmGsjhmHgqXcnH6Zz
2,0.816,0.600,6,-9.506,0,0.2530,0.1950,1.18e-02,0.1270,0.3890,⋯,https://api.spotify.com/v1/tracks/26EDvDMoB73VfJtT8pKTVS,https://api.spotify.com/v1/audio-analysis/26EDvDMoB73VfJtT8pKTVS,168425,4,100 Ks,Butch,no lyrics found,NA,NA,26EDvDMoB73VfJtT8pKTVS
3,0.525,0.753,1,-7.975,1,0.4990,0.4400,0.00e+00,0.0753,0.5520,⋯,https://api.spotify.com/v1/tracks/0aW657knJXm5pUjG7x6YN1,https://api.spotify.com/v1/audio-analysis/0aW657knJXm5pUjG7x6YN1,215466,4,Ground Zero,Tenk,no lyrics found,NA,NA,0aW657knJXm5pUjG7x6YN1
4,0.878,0.792,1,-5.483,1,0.0645,0.1810,0.00e+00,0.1190,0.5130,⋯,https://api.spotify.com/v1/tracks/5KZdZNWX2B0rLY7BEcgryb,https://api.spotify.com/v1/audio-analysis/5KZdZNWX2B0rLY7BEcgryb,161250,4,Bizele,Elinel,no lyrics found,NA,NA,5KZdZNWX2B0rLY7BEcgryb
5,0.453,0.656,5,-8.205,0,0.4250,0.2470,0.00e+00,0.0781,0.5660,⋯,https://api.spotify.com/v1/tracks/5lsGmbedEgu7ZxVuPByY1J,https://api.spotify.com/v1/audio-analysis/5lsGmbedEgu7ZxVuPByY1J,203625,4,K.P.T,Finem,no lyrics found,NA,NA,5lsGmbedEgu7ZxVuPByY1J


Constructing this data was a process. I'm not going to go into much detail, but the data collection was a four step process and happened in Python. 
1. [This page on everynoise.com](http://everynoise.com/thesoundsofcountries.html) has links to over 100 spotify playlists, each one specific to one country. Through a combination of webscraping and requests to the [Spotify Web API](https://developer.spotify.com/documentation/web-api/), information for all songs on all the playlist this page links to was requested. Those songs for which information was returned were made into a dataframe with columns each piece of their Spotify metadata.
2. Through many requests to the [Musixmatch API](https://developer.musixmatch.com/), only a small portion of which were fruitful, excerpts of the songs' lyrics were collected and placed into the data frame as well.
3. With the help of the [langdetect](https://github.com/Mimino666/langdetect) python library, the languages of a couple thousand of the lyric excerpts were able to be labeled as one of several dozen languages.

Some columns we don't need, like columns 5 and 16 among others. 

Those rows that for songs either whose lyrics couldn't be found or whose lyrics' language couldn't be detected will have NaNs in some columns. That won't do.

In [0]:
dat <- na.exclude(AFsongDF[,c(-5,-16:-12,-23)])
dat$sid <- as.character(dat$sid) #The songid column is a character, not a factor, so make sure that is reflected.

Get the encoding types of each lyric excerpt.

In [0]:
stri_lang_enc = vapply(1:nrow(dat), function(x){
  stri_enc_detect(
    dat$lyrics[x], filter_angle_brackets = F
    )[[1]][c('Language','Encoding','Confidence')][1,] %>%
    unlist()},
    # Language: What language does R think this text is most likely written in (if it has an opinion)
    # Encoding: What type of encoding does R think is most likely right for this text
    # Confidence: We probably won't use, but how confident is R that this is the correct language diagnosis
  FUN.VALUE = c('Language','Encoding','Confidence')) %>% t() %>%
  as.data.frame()

In [20]:
head(stri_lang_enc)

,Language,Encoding,Confidence
,<fct>,<fct>,<fct>
1,,UTF-8,1
2,,UTF-8,1
3,en,ISO-8859-1,0.26
4,,UTF-8,1
5,,UTF-8,0.8
6,pt,ISO-8859-1,0.31


Make the text doc df for corpus analysis by sifting through all the ones with weird encoding types, and keeping only the ones we can handle. 

Also, remove sketchy inconsistencies: It's sometimes common for songs in other languages to have parts in English as well. That's just going to make things confusing. So hopefull this will help a little with removing those.

In [17]:
lyrics_df2 <- dat %>%
  # columns need to be renamed as doc_id and text so that later, the corpus function will know what to do with the data we give it
  select(doc_id = sid, text = lyrics, lang) %>%
  # throw the encoding columns into it )
  cbind(stri_lang_enc) %>%
  mutate_if(is.factor, as.character) %>%
  # get rid of non-ISO encodings
  filter(Encoding %in% c("ISO-8859-1","ISO-8859-9","ISO-8859-2")) %>%
  # if lang.detect said English but R said something else was more likely, get rid of it
  dplyr::filter(ifelse(lang != "en", T, ifelse(lang == Language, T, F)))

langs <- data.frame(lang = c('en', 'es', 'id', 'sw', 'nl'), stringsAsFactors = F)

# only use the languages in the 5 specified
lyrics_df2 <- lyrics_df2 %>% 
  inner_join(langs)

# clean up
rm(langs)

head(lyrics_df2,2)

Joining, by = "lang"



,doc_id,text,lang,Language,Encoding,Confidence
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,5YPmo05u2Kd2HEGLFqaPXQ,Had lyom iss3id mbark ki wafitik a yamina ya taj lkhwdat khbark lmard libik hlkna w had lyom iss3id mbark ki wafitik a yamina ya taj lkhwdat khbark lmard libik hlkna mali tag 3lik nchofik we nssa9ssi f nass khyana mthawal galbi mndark kan lmola y9bl mna mali tag 3lik nchofik we nssa9ssi f nass khyana mthawal galbi mndark kan lmola y9bl mna hahahaa yamina w m3dm zinik al mglo3a hahahaa yamina w m3dm zinik a yamina blhkma chahrin nwdi hta dibt dwab lmlha kol lila w nhar nadi mayhlach nam lmha ra hobk ssfa mn jassdi w rabi 3ff w sabt raha nd3i lah ana njwjk ana chr3 w lh9 m3ana ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767),sw,hu,ISO-8859-2,0.18
2,5yqiverYlKEA0ly96HPq01,"Since 1992 there is a club which is making history. 7 years later, in 1999, it's still kicking: PONT AERI! When the stars begin to shine ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",en,en,ISO-8859-1,0.87


Take a smaller, stratified random sample. Limit to just five languages: English, Spanish, Indonesian, Dutch, and Swahili. Each strata is of 1 language and has 20 songs.

In [15]:
# take a smaller, evenly dispersed sample
set.seed(10)
## take an equal random sample from all five languages of interest
En_ids <- subset(x=lyrics_df2,subset = lang == 'en')$doc_id
## n = 20 because there are exactly 22 songs in Dutch, the language of the 5 with the fewest songs
En_ids <- sample(En_ids,20)
eS_ids <- subset(x=lyrics_df2,subset = lang == 'es')$doc_id
eS_ids <- sample(eS_ids,20)
Id_ids <- subset(x=lyrics_df2,subset = lang == 'id')$doc_id
Id_ids <- sample(Id_ids,20)
Nl_ids <- subset(x=lyrics_df2,subset = lang == 'nl')$doc_id
Nl_ids <- sample(Nl_ids,20)
sW_ids <- subset(x=lyrics_df2,subset = lang == 'sw')$doc_id
sW_ids <- sample(sW_ids,20)

# replace a misclassified song
En_ids_all <- subset(x=lyrics_df2,subset = lang == 'en')$doc_id
En_ids[which(En_ids == "720ZYTSr4vSqcFYq2CTJKN")] <-
  sample(En_ids_all[-which(En_ids_all %in% En_ids)],1)
rm(En_ids_all)
# replace another misclassified song
Id_ids_all <- subset(x=lyrics_df2,subset = lang == 'id')$doc_id
Id_ids[which(Id_ids == "7mrxKs2fqNiKBE8zePEP2l")] <-
  sample(Id_ids_all[-which(Id_ids_all %in% Id_ids)],1)
rm(Id_ids_all)

# make a data frame from the songs selected
esinw_df <- subset(x = lyrics_df2, select = names(lyrics_df2),
                   subset = doc_id %in% c(
                     En_ids, eS_ids, Id_ids, Nl_ids, sW_ids))
head(esinw_df,2)


,doc_id,text,lang,Language,Encoding,Confidence
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
69,5pz1Q9QFHWsUBZiJ73Jx3j,"Ze is niet altijd even vrolijk en dat ligt ook wel eens aan mij. En een beetje aan de weerman, maar die maakt eigenlijk niemand blij. En al zijn miezerige buien, daar heeft zij geen boodschap aan. Zij wil alle dagen zon, en als het moet eens een orkaan. Maar net als hem blijf ik proberen, elke dag een flauwe mop. Plots is daar dan toch die glimlach en dan klaart alles, dan klaart alles hier weer op. Want als ze lacht, breekt de hele hemel open, echt ik waan me in de tropen 't is echt machtig als ze lacht. Als ze lacht, baad ik uren in de zon ik wou dat ik dat voor haar kon wat zij voor mij doet als ze lacht. 'k Ben ook niet altijd even vrolijk, maar dat ligt echt wel niet aan mij. ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",nl,nl,ISO-8859-1,0.8
70,6eVj2ARRKQiy2Gb9Za7jnf,"Een lach, een groet, een blij gezicht Een vogel zwevend naar het licht. Oh het lijkt zo gewoon maar het is toch een wonder. Een kind dat lacht en naar je zwaait, een fietser die de hoek omdraait. Oh het lijkt zo gewoon maar het is toch een wonder. Het leven gaat zo snel voorbij, dat geld voor jou maar ook voor mij... Oh laat de zon in je hart Ze schijnt toch voor iedereen Geniet van het leven Want het duurt toch maar even. Oh aat de zon in je hart Ze schijnt toch voor iedereen Geniet van het leven ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",nl,nl,ISO-8859-1,0.91


Create a corpus object

In [21]:
# create a corpus of the lyrics
lyrics_corpus <- corpus(esinw_df)

# Let's get some information about our corpus, shall we?

# we don't care about sentences, song lyrics probably don't follow grammar protocols all that well
lyrics_sum <- summary(lyrics_corpus, tolower = T, n = 100) %>% dplyr::select(- Sentences)

# Lyric detail summary table
lyrics_sum3ds <- lyrics_sum %>% 
  mutate(Text = as.character(Text)) %>%
  mutate(TTR = Types/Tokens) %>% 
  mutate(Language = lang)

# get all the averages and overall totals for the metrics we have interest in
lyrics_sum_cct_avg <- lyrics_sum3ds %>% 
  summarise("Texts" = length(Text),
            "Avg Tokens per Lyric Excerpt" = round(mean(Tokens)),
            "Tokens in Total" = sum(Tokens),
            "Avg Types per Lyric Excerpt" = round(mean(Types),2), 
            "Avg Type/Token Ratio (TTR)" = round(mean(TTR),3)) %>% 
  mutate(Language = "all") %>%
  select(Language, Texts, `Tokens in Total`, `Avg Tokens per Lyric Excerpt`, `Avg Types per Lyric Excerpt`, `Avg Type/Token Ratio (TTR)`)

# get these ones again, but grouping by language this time
lyrics_sum_cct_va <- lyrics_sum3ds %>% group_by(Language) %>% 
  summarise("Texts" = length(Text),
            "Avg Tokens per Lyric Excerpt" = round(mean(Tokens)),
            "Tokens in Total" = sum(Tokens), 
            "Avg Types per Lyric Excerpt" = round(mean(Types),2), 
            "Avg Type/Token Ratio (TTR)" = round(mean(TTR),3)) %>%  ungroup() %>%
  select(Language, Texts, `Tokens in Total`, `Avg Tokens per Lyric Excerpt`, `Avg Types per Lyric Excerpt`, `Avg Type/Token Ratio (TTR)`)

# bind the language-grouped table and the everything table into one
lyrics_sum_cct_va <- rbind(lyrics_sum_cct_va, lyrics_sum_cct_avg)

# make a second version that only contains three details
lyrics_sum_cct_simp <- lyrics_sum_cct_va  %>%
  select(Language, `Avg Types per Lyric Excerpt`, `Avg Tokens per Lyric Excerpt`, `Avg Type/Token Ratio (TTR)`)

# Let's check it out
lyrics_sum_cct_va

Language,Texts,Tokens in Total,Avg Tokens per Lyric Excerpt,Avg Types per Lyric Excerpt,Avg Type/Token Ratio (TTR)
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
en,20,2543,127,68.10,0.544
es,20,2167,108,59.70,0.559
id,20,1528,76,47.70,0.625
nl,20,2569,128,71.15,0.559
sw,20,2229,111,61.85,0.560
all,100,11036,110,61.70,0.569
